In [1]:
import sys
sys.path.append("../")
from fns import actions, isolate_action_data, messages_prompt
from IPython.display import Markdown as md


def action_to_prompt(actions):
    stripped = [isolate_action_data(action) for action in actions]
    descriptions = ""

    stripped.sort(key=lambda x: x['name'])

    action_strings = {}
    for action in stripped:
        if action['name'] not in action_strings:
            action_strings[action['name']] = []
        action_strings[action['name']].append(action['description'])

    for action in action_strings:
        descriptions += f"- {action}: has the following features:\n"
        for description in action_strings[action]:
            descriptions += f"    - {description}\n"

    return descriptions


def planner_prompt(actions):
    return f"""
You are a helpful assisant. 


Task:

Given a user's request, your job is to determine a comprehensive plan of action
that will result in the resolution of the request. Take as many actions as you
need to address the user's information needs so that the end result is complete,
accurate and tailored to the user's inquiry.

If multiple actions are required, list them in order so that outputs of a prior
action can serve as inputs to the following one. Be as thorough as required.

Available Actions: 

{action_to_prompt(actions)} 

Formatting:

Please format your response as bullets of actions, in the order you would take
them, with the following format:

- action_1: one sentence reason for action 1
- action_n: one sentence reason for action n+1

> Note: You can also choose to take no action if you don't know what to do. 

"""


system_msg = {
    "role": "system",
    "content": planner_prompt(actions)
}

md(planner_prompt(actions))


You are a helpful assisant. 


Task:

Given a user's request, your job is to determine a comprehensive plan of action
that will result in the resolution of the request. Take as many actions as you
need to address the user's information needs so that the end result is complete,
accurate and tailored to the user's inquiry.

If multiple actions are required, list them in order so that outputs of a prior
action can serve as inputs to the following one. Be as thorough as required.

Available Actions: 

- action_vtx_api: has the following features:
    - Takes actions against one of Vertex's APIs.
    - Executes a command on a user's behalf.
    - Not for answering questions. Only for executing commands.
    - Executes a command on a user's behalf with specific information. I do not answer questions.
- ask_user: has the following features:
    - When the assistant needs more information from the user to proceed. Not for commands.
    - Ends a conversation. No actions to follow.
- elaborate_vtx_url: has the following features:
    - Takes a single URL and returns the full text of the page.
    - Can elaborate on a webpage (for a URL). Not for summarization.
    - Returns the full content of an entire web page.
    - Get's all the context surrounding a particular topic.
- llm_eloquent: has the following features:
    - Summarizes text to make it more concise and tailored to the user's request.
    - Summarizes a page.
    - Rephrases a procured chunk of text to address their question as a helpful response.
    - When text needs to be summarized, I can pull out only the relevant information.
- llm_pick: has the following features:
    - When a list is provided, this action picks the item that is most relevant to the user's request.
    - When a list of URLs is available, picks one that is most relevant.
    - Picks an item from a list that best suits a user's query.
    - Does not provide the full text of the result, just an identifier.
- search_vtx_com: has the following features:
    - For general inquiries about Vertex's products or services. Returns a list of search results.
    - Questions about sales and marketing materials.
    - Not for information on a specific Vertex product.
    - Returns a list of search results of pages with only the following attributes per result: title, URL, and snippet.
- search_vtx_kb: has the following features:
    - Search for specific/esoteric information about a Vertex product/service.
    - Returns a list of search results, each containing completed chunks of knowledge.
    - Returns a list containing results that each have more detailed information about Vertex's products and services. Typically don't require elaboration.
 

Formatting:

Please format your response as bullets of actions, in the order you would take
them, with the following format:

- action_1: one sentence reason for action 1
- action_n: one sentence reason for action n+1

> Note: You can also choose to take no action if you don't know what to do. 



In [2]:
def test_query(content):
    res = messages_prompt([system_msg, {
        "role": "user",
        "content": content
    }])
    return res

In [3]:

# expected output:
# - search_vtx_com
# - llm_pick
# - elaborate_vtx_url
# - llm_eloquent

test_1 = test_query("How do I contact Vertex Inc customer service?")

print(test_1)

- search_vtx_com: To find general information about how to contact Vertex Inc's customer service.


In [4]:
# expected output:
# - action_vtx_api

test_2 = test_query("Add a taxpayer to my account.")
print(test_2)

- action_vtx_api: To add a taxpayer to the user's account, we need to execute this command through Vertex's API.


In [5]:

# expected output:
# - search_vtx_kb
# - llm_pick
# - llm_eloquent


test_3 = test_query("What are the features of O Series?")
print(test_3)

- search_vtx_kb: To find specific and detailed information about the features of O Series from Vertex's knowledge base.
